In [1]:
from selenium import webdriver
import pandas as pd
from scrap_article_current_tab import scrap_article

from datetime import datetime as dt
from datetime import timedelta as td
from os import makedirs
from os.path import exists
from time import sleep
from json import loads

In [2]:
now = dt.now()
year = now.year
month = now.month
day = now.day
ts = str(int(round(now.timestamp())))

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
chrome = webdriver.Chrome(options=options)

In [3]:
# chrome = webdriver.Chrome()

In [4]:
chrome.get(url = 'http://wyborcza.pl/0,75248.html?str=100_23719317')

In [5]:
sleep(10)

In [6]:
# try to click rodo (first click opens sometimes an ad also)
for i in range(3):
    try:
        sleep(3)
        chrome.find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[1]/div/p[2]')\
            .click()
    except:
        pass

In [7]:
# close new window if exists
try:
    chrome.switch_to.window(chrome.window_handles[1])
    chrome.close()
except:
    pass

In [8]:
chrome.switch_to.window(chrome.window_handles[0])

In [9]:
chrome.get('https://login.wyborcza.pl/')

In [16]:
chrome\
    .find_element_by_xpath('//*[@id="wyborczaEmail"]')\
    .send_keys('slusarczyk1@wp.pl')

In [19]:
chrome\
    .find_element_by_xpath('//*[@id="wyborczaPassword"]')\
    .send_keys('Sraniejebanko1')

In [20]:
chrome\
    .find_element_by_xpath('/html/body/section/section[1]/form/div[4]/button')\
    .click()

In [21]:
sleep(5)

In [22]:
urls = []
for i in range(100, 1000):
    chrome.get(url = 'http://wyborcza.pl/0,75248.html?str='+str(i)+'_23719317')
    for entry in chrome\
        .find_element_by_xpath('//*[@id="pagetype_index"]/section[5]/div/main/div/div[2]/div/div[2]/ul')\
        .find_elements_by_class_name('entry'):
        
        url = entry.find_element_by_tag_name('h3')\
                .find_element_by_tag_name('a')\
                .get_attribute('href')
        #scrap_article(chrome, url, first_time)
        urls.append(url)
    break
    

In [24]:
import pandas as pd

In [25]:
# df = pd.DataFrame(urls, columns = ['urls'])

In [26]:
# df.to_csv('old_urls.csv')

In [27]:
df = pd.read_csv('old_urls.csv')

In [28]:
errors = []

In [ ]:
for url in df['urls'].values:
    print(url)
    try:
        a = scrap_article(chrome, url, True)
        with open('data/comments/'+a['url'].split('/')[-1].split(',')[2], 'w') as f:
            f.write(str({'row': {
               'comments': a,
               'timestamp': dt.timestamp(dt.now())
               }
            }))
    except:
        errors.append(url)
    print(a['url'].split('/')[-1].split(',')[2])

In [30]:
a

{'url': 'http://krakow.wyborcza.pl/krakow/7,44425,24667232,radni-starego-miasta-chca-zatrudnic-sokolnika-do-ploszenia-golebi.html',
 'division': 'Stare Miasto',
 'pub_date': '16 kwietnia 2019 | 14:41',
 'author': 'Aleksander Gurgul',
 'title': 'Radni Starego Miasta chcą zatrudnić sokolnika do płoszenia gołębi. Ornitolodzy: "Nonsens"',
 'highlight': 'Radni ze Starego Miasta chcą, by prezydent Krakowa zatrudnił sokolnika, który np. na Plantach będzie odławiał niechciane ptaki, w tym gołębie. - To tak jakby zatrudnić na etacie osobę z packą na muchy, której zadaniem by było tępienie owadów - odpowiadają na ich pomysł ornitolodzy.',
 'content': 'Radni Dzielnicy I na najbliższej sesji zajmą się projektem uchwały, w której proszą prezydenta Krakowa o zatrudnienie sokolnika. Miałby on pracować w strukturach Zarządu Zieleni Miejskiej i pomagać w ogrodzie zoologicznym.„Celem działania sokolnika miejskiego byłoby przepłaszanie dużych ptaków (kruki, gawrony, gołębie), które w ostatnich latach szc

In [ ]:
pd.DataFrame(errors, columns = ['urls']).to_csv('error_urls.csv')